In [1]:
import pandas as pd

import pdb
from IPython.core.display import display #display inline package
from datetime import timedelta

import datetime

from scipy import stats
import numpy as np
from scipy.stats.stats import pearsonr

#from sklearn.cross_validation import KFold

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor

from scipy.misc import derivative #derivates of functions tool
from scipy.interpolate import UnivariateSpline #basic spline fitting tool
from scipy.signal import gaussian #gaussian filtering tool
from scipy.ndimage import filters #anotehr filtering tool
from scipy import stats #core scipy stats package
from scipy.integrate import simps #simpsons rule package for trapezoid calcualtions in area under curve calcs


import plotly.tools as tls
tls.set_credentials_file(username='NTPlotly', api_key='yViUde2xMRvWglgycSg4') #my plotly credentials (please dont use they cost me $$)
import plotly.plotly as py
#from plotly.graph_objs import *
import plotly.graph_objs as go
from plotly import tools
#import cufflinks as cf


In [2]:
#function to get the data and split into peaks, troughs and all.

def getData(fileName):
    
    df = pd.read_csv(fileName)

    df['dates'] = pd.to_datetime(df['dates'])
    
    peaks = df.loc[(df['peaks']==True)] 
    
    troughs = df.loc[(df['troughs']==True)] 
    
    return peaks, troughs, df
    

In [3]:
def doLinearModel(data):
    
    x = range(1,len(data)+1)
    
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,data)
    
    return slope,intercept

In [4]:
def doStd(sliced):
    
    
    unDates = sliced.dates.unique()
    
    outStd = []
    
    outDf = pd.DataFrame()   
    
    #pdb.set_trace()
    
    for i in unDates:
        
        timeSlice = sliced[(sliced['dates'] == i)]
        
        #std = timeSlice['fittedSpline'].std()
        
        std = timeSlice['fittedSpline'].std()
        
        outStd.append(std)
        
        #pdb.set_trace()
     
    outDf['dates'] = pd.to_datetime(unDates)
    
    outDf['std'] = outStd
    
    sortedDf = outDf.sort_values(by='dates',ascending=True)
        
    return sortedDf
    

In [5]:
def doAvg(sliced):
    
    
    unDates = sliced.dates.unique()
    
    outStd = []
    
    outDf = pd.DataFrame()   
    
    #pdb.set_trace()
    
    for i in unDates:
        
        timeSlice = sliced[(sliced['dates'] == i)]
        
        #std = timeSlice['fittedSpline'].std()
        
        std = timeSlice['fittedSpline'].median()
        
        outStd.append(std)
        
        #pdb.set_trace()
     
    outDf['dates'] = pd.to_datetime(unDates)
    
    outDf['std'] = outStd
    
    sortedDf = outDf.sort_values(by='dates',ascending=True)
        
    return sortedDf

In [6]:
def doDateDiff(troughs):
    
    diffs = []
    
    for i in range(len(troughs)):
        
        
        if i <len(troughs)-1:
            
            d1 = troughs['troughs'].loc[i]
            
            d2 = troughs['troughs'].iloc[i+1]
            
            dif = d2 - d1
            
            diffs.append(dif)
            
            #pdb.set_trace()
            
            
    return diffs  
        

In [146]:
def doTimeSeries(peakMajor,troughMajor,df):
    
    #pdb.set_trace()
    
    #change medians to peaks
    
    #startPeak = peakMajor['peaks'].iloc[0]
    
    #startTrough = troughMajor['troughs'].iloc[0]
    
    troughStd = []
    
    peakStd = []
    
    peakSlopes = []
    
    troughSlopes = []
    
    troughLengths = []
    
    peakLengths = []
    
    peakPlots = []
    
    troughPlots = []
    
    #trough to peaks
    
    for i in range(len(troughMajor)):
        
        #pdb.set_trace()
    
        troughDate = troughMajor['troughs'].iloc[i]
                        
        greater = peakMajor[(peakMajor['peaks'] > troughDate)]
        
        if greater.empty == False:
        
            peakDate = greater['peaks'].iloc[0]

            sliced = df[(df['dates'] > troughDate) & (df['dates'] <= peakDate)]
            
            std = doStd(sliced)
            
            stdNotNull = pd.notnull(std)
            
            #pdb.set_trace()
            
            std = std[stdNotNull['std']]
            
            troughLengths.append(len(std))
            
            slope, intercept = doLinearModel(std['std'].tolist())
            
            troughSlopes.append(slope)  
            
            xi = np.arange(0,len(std))
            
            #pdb.set_trace()
            
            line = slope*xi+intercept
            
            if i == 1:
                
                lg = True
                
            else:
                
                lg = False
                    
            linear = go.Scatter(
                  x=std['dates'],
                  y=line,
                  mode='lines',
                  line = dict(width=2,color = 'rgb(0,0,255)'),
                  name='Slope',
                  #yaxis = 'y2',
                  showlegend = lg
                  )

            troughPlots.append(linear)

            '''
            
            result,plot = doDistance(sliced)            
            
            plot = plot[0:100]
            
            #pdb.set_trace()
            
            title = 'Comparison plots'
                
            layout = go.Layout(title=title,showlegend=False,yaxis=dict(title='Fitted Non-Photosynthetic Vegetation Cover %',range=[0, 100]),xaxis=dict(title='Time'),font=dict(family='Arial, monospace', size=14, color = ('rgb(1, 1, 1)')))
      
            fig = go.Figure(data=plot, layout=layout)
        
                            
            filename = 'outputs/'+ 'TtoP'+ '_' + str(i) + '.png'
            
            #url = py.iplot(fig,filename=filename)
            
            #pdb.set_trace()
            
            py.image.save_as(fig, filename=filename)
            
            #pdb.set_trace()
            
            '''

            avgStd = np.mean(std)
            
            troughStd.append(avgStd) 
            
      
    #pdb.set_trace()
            
    #peaks to troughs

    for i in range(len(peakMajor)):    
        
            #print('peaks' + str(i))
            
            #i = 4  
            
            #pdb.set_trace()
            
            peakDate = peakMajor['peaks'].iloc[i]
            
            #pdb.set_trace()
            
            greater = troughMajor[(troughMajor['troughs'] > peakDate)]
            
            
            if greater.empty == False:
                
                troughDate = greater['troughs'].iloc[0]
            
                sliced = df[(df['dates'] > peakDate) & (df['dates'] <= troughDate)]
                                
                std = doStd(sliced)
            
                stdNotNull = pd.notnull(std)            
            
                std = std[stdNotNull['std']]
            
                peakLengths.append(len(std))
            
                slope, intercept = doLinearModel(std['std'].tolist())
            
                peakSlopes.append(slope)
                
                xi = np.arange(0,len(std))
            
                line = slope*xi+intercept
                
                if i == 1:

                    lg = True

                else:

                    lg = False
                    
                linear = go.Scatter(
                      x=std['dates'],
                      y=line,
                      mode='lines',
                      line = dict(width=2,color = 'rgb(0,0,255)'),
                      name='Peak to trough slope',
                      #yaxis = 'y2',
                      showlegend = False
                      )

                peakPlots.append(linear)
                
                '''
                
                result,plot = doDistance(sliced)
                
                plot = plot[0:100]
                
                title = 'Comparison plots'
                
                layout = go.Layout(title=title,showlegend=False,yaxis=dict(title='Fitted Non-Photosynthetic Vegetation Cover %'),xaxis=dict(title='Time'),font=dict(family='Arial, monospace', size=14, color = ('rgb(1, 1, 1)')))
      
                fig = go.Figure(data=plot, layout=layout)
        
                            
                filenameClimate = 'outputs/'+ 'PtoT'+ '_' + str(i) + '.png'
            
                py.image.save_as(fig, filename=filenameClimate)
        
                   
                #pdb.set_trace()
                
                avgStd = np.mean(std)
            
                peakStd.append(avgStd) 

                '''

    return peakStd, troughStd,peakSlopes,troughSlopes,troughLengths, peakLengths, troughPlots, peakPlots
    
    
    

In [147]:
def doStdPlots(outDf):
    

    #trace = go.Scatter(x = outDf['dates'],y = outDf['std'],mode = 'markers',yaxis = 'y2',name='Standard Deviation',marker = dict(color = ('rgb(128, 128, 128)'),size = 4,symbol='circle'),showlegend = True)
    
    trace = go.Scatter(x = outDf['dates'],y = outDf['std'],mode = 'markers',name='Standard Deviation',marker = dict(color = ('rgb(128, 128, 128)'),size = 3,symbol='circle'),showlegend = True)
    
    
    data = [trace]
   

    title = 'Standard deviation'
    
   
    layout = go.Layout(title=title,showlegend=False,yaxis=dict(title='Fitted Non-Photosynthetic Vegetation Cover %'),xaxis=dict(title='Time'),font=dict(family='Arial, monospace', size=8, color = ('rgb(1, 1, 1)')))
        
        
        
    fig = go.Figure(data=data, layout=layout)        
                            
    #filename = '~/work/outputs/stdAll.png'
            
    #py.image.save_as(fig, filename=filename)
    
    return trace
        

In [148]:
#simple function to call moving average function
def doParameters(y):
        
    avg, param = movingAverage(y)
           
    return param 
    

In [149]:
#Gaussian type filter to find both the average and the variance of the raw data.
def movingAverage(series, sigma=3):

    b = gaussian(39, sigma)
    
    average = filters.convolve1d(series, b/b.sum())
    
    var = filters.convolve1d(np.power(series-average,2), b/b.sum())
    
    return average, var

In [150]:
def doSpline(x,y,paramVar):
    
    #pdb.set_trace()
    
    spl1 = UnivariateSpline(x, y, w=0.9/np.sqrt(paramVar)) # weighting factor method
    #spl1 = UnivariateSpline(x, y) # weighting factor method   
    
    #spl2 = UnivariateSpline(x, y,s=paramMan) #manual smoothing method
    
        
    return spl1

In [151]:
def findDerivatives(spl,x):
    
    splDeriv = []
       
    for i in x:

        a = spl.derivatives(i)

        splDeriv.append(a[1])
    
    
    return splDeriv

In [152]:
def crossingTest(first):

    peakCrossing = np.zeros(len(first))

    troughCrossing = np.zeros(len(first))

    for i in range(len(first)):

        if i >0:

            d1 = first[i-1]

            d2 = first[i]
            
            d1s = sign(d1)

            d2s = sign(d2)

            if d1s > d2s:

                peakCrossing[i]=1

            elif d1s < d2s:

                troughCrossing[i]=1

    return peakCrossing, troughCrossing

In [153]:
#basic sign test
def sign(x): 

    return 1 if x >= 0 else -1

In [195]:
def doAll():
    
    timeSeriesList = pd.read_csv('~/work/data/list.txt',header=None)
    
    #pdb.set_trace()
    
    allTroughSlopes = []
    
    allPeakSlopes = []
    
    allSiteIds = []    
    
    allTroughObs = []
    
    allPeakObs = []
    
    allTroughLengths = []
    
    allPeakLengths = []
    
    names = ['Semi-arid to Arid Introduced Grassland','Arid to Semi-arid Chenopod Shrub-land and Grassland','Semi-arid to Arid Spinifex (Trioida sp.) Grassland','Sub-tropical to Semi-arid Grassland']
   

    for i in range(len(timeSeriesList)):
            
        print i
        
        #i = 1
        
        name = names[i]
        
        fileName = timeSeriesList.loc[i]
        
        fileName = fileName[0]
        
        fileName = '~/work/data/' + fileName
        
        obs = fileName[22:-4]
        
        peaks, troughs, df = getData(fileName) #get the sites timeseries data

        outDf = doStd(df)# calculate the standard deviation of all the timeseries 
        
        avg = doAvg(df)
        
        
        param = doParameters(avg['std'])
        
        x = range(len(avg))
        
        smoothed = doSpline(x,avg['std'],param)
        
        splDerivVar = findDerivatives(smoothed,x) 
        
        peakCrossingVar,troughCrossingVar = crossingTest(splDerivVar)
        
        spline = smoothed(x)
        
        spline = spline-100
        
        peaksVar = peakCrossingVar == 1
        
        troughsVar = troughCrossingVar == 1 
        
        #pdb.set_trace()
        
        peakDates =  avg[peaksVar]
        
        peakValues = spline[peaksVar]
                               
        troughDates =  avg[troughsVar]
        
        troughValues = spline[troughsVar]
       
        '''
        peaks = go.Scatter(x=peakDates['dates'], y=peakValues,name='Peaks',mode = 'markers', marker = dict(color = ('rgb(1,1,1)'),size = 13,symbol='circle'))
        
        troughs = go.Scatter(x=troughDates['dates'], y=troughValues,name='troughs',mode = 'markers', marker = dict(color = ('rgb(1, 1, 1)'),size = 13,symbol='triangle-up'))
               
        
        
        fitted = go.Scatter(x=avg['dates'], y=spline,name='NPV Spline',mode = 'lines', line = dict(color = ('rgb(1, 1, 1)'),width = 1))
           
            
            
        data = [peaks,troughs,fitted]
        
        #py.iplot(data, filename='basic-scatter')
        
        
        
        layout = go.Layout(title='test',yaxis=dict(title='Non-Photosynthetic Vegetation Cover %',showgrid=False),xaxis=dict(title='Time',showgrid=False),font=dict(family='Arial, monospace', size=30, color = ('rgb(1, 1, 1)')))
      
        fig = go.Figure(data=data, layout=layout)
        
        filename1 = '~/work/data/test'
        
        peakPlot = py.iplot(fig, filename=filename1)
        
        #display(peakPlot)
        
        
        
        '''

        
        
        newPeaks = pd.DataFrame()
        
        newTroughs = pd.DataFrame()
        
        newPeaks['peaks'] = peakDates['dates']
        
        newPeaks = newPeaks.reset_index(drop=True)
        
        newTroughs['troughs'] = troughDates['dates']
        
        newTroughs = newTroughs.reset_index(drop=True)
        
        #pdb.set_trace()

        peakStd, troughStd,peakSlopes,troughSlopes,troughLengths, peakLengths,troughPlots, peakPlots = doTimeSeries(newPeaks,newTroughs,df)

        for ii in range(len(troughSlopes)):                          
                
            allTroughSlopes.append(troughSlopes[ii])
                    
            allTroughObs.append(obs)
                    
            allTroughLengths.append(troughLengths[ii])
                    
        for ii in range(len(peakSlopes)):
                    
            allPeakSlopes.append(peakSlopes[ii])
                    
            allPeakObs.append(obs)
                    
            allPeakLengths.append(peakLengths[ii])

        #pdb.set_trace()
            
        stdPlot = doStdPlots(outDf)

        doallPlots(df,newPeaks,newTroughs,stdPlot,troughPlots, peakPlots,obs,peakSlopes,troughSlopes,name)

           
    #pdb.set_trace()   
    
        
    outputPeaks = pd.DataFrame()
    
    outputTroughs = pd.DataFrame()
    
    outputPeaks['obsPeaks'] = allPeakObs
    
    outputTroughs['obsTroughs'] = allTroughObs
    
    outputPeaks['peaks'] = allPeakSlopes
    
    outputTroughs['troughs'] = allTroughSlopes
    
    outputTroughs['troughlengths'] = allTroughLengths
    
    outputPeaks['peaklengths'] = allPeakLengths
    
    outputTroughs.to_csv('timeseriesSlopesTroughs.csv')
    
    outputPeaks.to_csv('timeseriesSlopesPeaks.csv')
    
   

In [280]:
def doallPlots(df,peakMajor,troughMajor,stdPlot,peakPlots,troughPlots,fileName,peakSlopes,troughSlopes,name):
    
    plots = []
   
    peakShapes = []
    
    troughShapes = []
    
    #pdb.set_trace()
    
    
    peakStart = peakMajor['peaks'].iloc[0]
    
    peakEnd = peakMajor['peaks'].iloc[-1]
    
    troughStart = troughMajor['troughs'].iloc[0]
    
    troughEnd = troughMajor['troughs'].iloc[-1]
    
    start = peakStart - troughStart
    
    end = peakEnd - troughEnd
    
    peakAnnotate = []
    
    troughAnnotate = []
    
    annotations = []
    
    if start.days <0:
         
        for i in range(len(peakMajor)):
            
            case = {'type': 'rect', 'x0': peakMajor['peaks'].iloc[i], 'x1':troughMajor['troughs'].iloc[i],'y0':0,'y1':30,'fillcolor': 'rgb(155, 155, 155)','opacity': 0.1,'line': {'width': 0,} }

            peakShapes.append(case)
            
            avgDate = peakMajor['peaks'].iloc[i] + (troughMajor['troughs'].iloc[i] - peakMajor['peaks'].iloc[i])/2
                
                
            #pdb.set_trace()
            
            pSlope = round(peakSlopes[i],3)
                
            annotations.append(dict(x=avgDate,y=20,xref='x',yref='y',text = str(pSlope),showarrow=False,ax=0,ay=-80,font=dict(size=14)))
         
        
                
            #peakAnnotate.append(trace)

        for i in range(len(troughMajor)):


            if i < len(peakMajor)-1:

                case = {'type': 'rect', 'x0': troughMajor['troughs'].iloc[i], 'x1':peakMajor['peaks'].iloc[i+1],'y0':0,'y1':30,'fillcolor': 'rgb(100, 100, 100)','opacity': 0.1,'line': {'width': 0,} }

                troughShapes.append(case)
                
                avgDate = peakMajor['peaks'].iloc[i+1] + (troughMajor['troughs'].iloc[i] - peakMajor['peaks'].iloc[i+1])/2
            
                tSlope = round(troughSlopes[i],3)
                
                annotations.append(dict(x=avgDate,y=22,xref='x',yref='y',text = str(tSlope),showarrow=False,ax=0,ay=-80,font=dict(size=14)))
         
                
    else:
        
        
           
        for i in range(len(peakMajor)):
            
            
            if i < len(troughMajor)-1:

                case = {'type': 'rect', 'x0': peakMajor['peaks'].iloc[i], 'x1':troughMajor['troughs'].iloc[i+1],'y0':0,'y1':30,'fillcolor': 'rgb(155, 155, 155)','opacity': 0.1,'line': {'width': 0,} }

                peakShapes.append(case)
                
                avgDate = peakMajor['peaks'].iloc[i] + (troughMajor['troughs'].iloc[i+1] - peakMajor['peaks'].iloc[i])/2
                
                
                #pdb.set_trace()
                
                pSlope = round(peakSlopes[i],3)
                
                annotations.append(dict(x=avgDate,y=20,xref='x',yref='y',text = str(pSlope),showarrow=False,ax=0,ay=-80,font=dict(size=14)))
         
    
                
                #pdb.set_trace()

        for i in range(len(troughMajor)):
            
            if i <= len(peakMajor)-1:

                case = {'type': 'rect', 'x0': troughMajor['troughs'].iloc[i], 'x1':peakMajor['peaks'].iloc[i],'y0':0,'y1':30,'fillcolor': 'rgb(100, 100, 100)','opacity': 0.1,'line': {'width': 0,} }

                tSlope = round(troughSlopes[i],3)
                
                avgDate = peakMajor['peaks'].iloc[i] + (troughMajor['troughs'].iloc[i] - peakMajor['peaks'].iloc[i])/2
                
                annotations.append(dict(x=avgDate,y=22,xref='x',yref='y',text = str(tSlope),showarrow=False,ax=0,ay=-80,font=dict(size=14)))
                                
                troughShapes.append(case)

    #pdb.set_trace()
    
    #case = {'type': 'rect','xref': 'canvas','yref': 'canvas', 'x0': 102, 'x1':104,'y0':61,'y1':71,'fillcolor': 'rgb(255, 155, 155)','opacity': 0.2,'line': {'width': 0,} }

    #peakShapes.append(case)
    
    allShapes = peakShapes + troughShapes
    
    
    for i in range(66):
            
            for ii in range(66):
                
                pixAll = df[(df['x'] == i) & (df['y'] == ii)]

                yA = pixAll['fittedSpline'].tolist()

                datesA = pixAll['dates']

                yA = np.array(yA) -100

                xA = range(1,len(yA)+1)

                #x = list(x)

                xA = pd.Series(xA)
                
                if i == 0 | ii == 0:
                    
                    lgnd = True
                
                else:
                    
                    lgnd = False
                    
                #pdb.set_trace()

                plotAll = go.Scatter(name = 'NPV', x=datesA, y=yA,mode = 'lines',line = dict(width=1,color = 'rgb(0,0,0)'),showlegend = lgnd)        

                plots.append(plotAll)

    title = name
    
    plotSubset = plots[0:100]
    
                 
    #layout = go.Layout(title=title,shapes = allShapes, showlegend=False,yaxis=dict(title='Fitted Non-Photosynthetic Vegetation Cover %',range=[0, 100]),xaxis=dict(title='Time'),font=dict(family='Arial, monospace', size=14, color = ('rgb(1, 1, 1)')))
      
    #layout = go.Layout(title=title,showlegend=False,shapes = allShapes,yaxis2=dict(title='Standard Deviation',side='right',overlaying='y',range=[-10,13]),yaxis=dict(title='Fitted Non-Photosynthetic Vegetation Cover %',range=[0, 100]),xaxis=dict(title='Time'),font=dict(family='Arial, monospace', size=14, color = ('rgb(1, 1, 1)')))
    
    
    topPlot = []
    
    bottomPlot = []
    
    topPlot.append(stdPlot)
    
    
    
    for i in range(len(peakPlots)):
                
        
        topPlot.append(peakPlots[i])
        
   
    #for i in range(len(peakAnnotate)):
        
        #topPlot.append(peakAnnotate[i])
    
    #trace0 = go.Scatter(x=troughMajor['troughs'].iloc[1],y=20,text='Line positioned relative to the plot',mode='text',)
      
    
    
    for i in troughPlots:
        
        topPlot.append(i) 
        
            
    for i in plotSubset:
        
        bottomPlot.append(i)
        
    fig = tls.make_subplots(rows=2, cols=1,shared_xaxes=True)
    
    
    
    #fig.append_trace(topPlot, 1, 1)
    
    for ii in bottomPlot:
        
        fig.append_trace(ii, 2, 1) 
        
    for ii in topPlot:
        
        fig.append_trace(ii, 1, 1)
        
    
    
    #fig['layout'].update(title=title,showlegend=False,shapes = allShapes,yaxis=dict(title='Fitted Non-Photosynthetic Vegetation Cover %',range=[0, 100]),xaxis=dict(title='Time'),font=dict(family='Arial, monospace', size=14, color = ('rgb(1, 1, 1)')))

    fig['layout'].update(height=800, width=2200, title=title,shapes = allShapes, annotations = annotations,titlefont=dict(size=35,color='rgb(0,0,0)'))
    
    fig['layout']['xaxis1'].update(title='Time',titlefont=dict(size=30,color='rgb(0,0,0)'))
    fig['layout']['yaxis1'].update(title='Standard deviation',titlefont=dict(size=30,color='rgb(0,0,0)'))
    fig['layout']['yaxis2'].update(title='Fitted NPV',titlefont=dict(size=30,color='rgb(0,0,0)'))
    
    #case = {'type': 'rect','xref': 'paper','yref': 'paper', 'x0': 102.44, 'x1':104.26,'y0':65.81,'y1':70.97,'fillcolor': 'rgb(255, 155, 155)','opacity': 0.2,'line': {'width': 0,} }

    #shp = [case]
    
    #fig['layout'].update(shapes = shp)
    
                                    
    filename =  fileName + '_' + '.png'
    
    peakPlot = py.iplot(fig, filename=filename)
    
    
    
    #display(peakPlot)
            
    py.image.save_as(fig, filename=filename)
    
    pdb.set_trace()
    
    

In [ ]:
doAll()

0
This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]

The draw time for this plot will be slow for clients without much RAM.
--Return--
> <ipython-input-280-7d4d8ba1b155>(230)doallPlots()->None
-> pdb.set_trace()


KeyboardInterrupt: 